In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 542M
6.4M  twic1417.pgn
6.8M  twic1417_moves.csv
 32K  twic1417_game_info.csv
4.5M  twic1407.pgn
 13M  twic1407_moves.csv
 72K  twic1407_game_info.csv
 16K  README.md
 89M  lichess_db_standard_rated_2013-01.pgn
286M  lichess_db_standard_rated_2013-01_moves.csv
 33M  lichess_db_standard_rated_2013-01_game_info.csv
568K  chess.ipynb
184K  chess-and-eigenvectors.ipynb
4.0K  caruana_carlsen_2018.pgn
160K  caruana_carlsen_2018_moves.csv
4.0K  caruana_carlsen_2018_game_info.csv
6.7M '2021-09.bare.[6516].pgn'
 43M '2021-09.bare.[6516]_moves.csv'
136K '2021-09.bare.[6516]_game_info.csv'
 11M '2021-01.bare.[10357].pgn'
 44M '2021-01.bare.[10357]_moves.csv'
152K '2021-01.bare.[10357]_game_info.csv'


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
state_dict

{'a1': 'a1',
 'b1': 'b1',
 'c1': 'c1',
 'd1': 'd1',
 'e1': 'e1',
 'f1': 'f1',
 'g1': 'g1',
 'h1': 'h1',
 'a2': 'a2',
 'b2': 'b2',
 'c2': 'c2',
 'd2': 'd2',
 'e2': 'e2',
 'f2': 'f2',
 'g2': 'g2',
 'h2': 'h2',
 'a3': None,
 'b3': None,
 'c3': None,
 'd3': None,
 'e3': None,
 'f3': None,
 'g3': None,
 'h3': None,
 'a4': None,
 'b4': None,
 'c4': None,
 'd4': None,
 'e4': None,
 'f4': None,
 'g4': None,
 'h4': None,
 'a5': None,
 'b5': None,
 'c5': None,
 'd5': None,
 'e5': None,
 'f5': None,
 'g5': None,
 'h5': None,
 'a6': None,
 'b6': None,
 'c6': None,
 'd6': None,
 'e6': None,
 'f6': None,
 'g6': None,
 'h6': None,
 'a7': 'a7',
 'b7': 'b7',
 'c7': 'c7',
 'd7': 'd7',
 'e7': 'e7',
 'f7': 'f7',
 'g7': 'g7',
 'h7': 'h7',
 'a8': 'a8',
 'b8': 'b8',
 'c8': 'c8',
 'd8': 'd8',
 'e8': 'e8',
 'f8': 'f8',
 'g8': 'g8',
 'h8': 'h8'}

In [8]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    counter = 0
    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

    #print(game)
    #do stuff here



    #for each line in the file
    for move in game.mainline_moves():
        counter += 1
        if board.is_pseudo_legal(move):


            if board.is_capture(move):
                capturing_piece = state_dict[chess.square_name(move.from_square)]
                captured_piece = state_dict[chess.square_name(move.to_square)]
                #print('capturing piece',capturing_piece)
                #print('captured piece',captured_piece)

                if board.is_en_passant(move):
                    #do en passant stuff here
                    print("en passant")
                elif capturing_piece is not None:
                    if captured_piece is not None:

                        #record the kill in a matrix of all kills
                        #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                        #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                        #what piece is there now?
                     

                        #increment an entry in the weights matrix when one piece takes another
                        weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                        #update the piece on the target square
                        state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                        #update the source square to be empty
                        state_dict[chess.square_name(move.from_square)] = None

                        #board.push(move)  # Make the move


                        #if df.loc[i+1,'move_no'] == 1:
                        #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            else:


                #add castling special cases to record both moves
                if board.is_kingside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles kingside')
                        #move the appopriate rook
                        state_dict['f1'] = 'h1'
                        #leaving the rook source square empty
                        state_dict['h1'] = 'None'
                        #board.push(move)  # Make the move



                    elif not board.turn:
                        #print(counter,'... and black castles kingside')
                        #move the appopriate rook
                        state_dict['f8'] = 'h8'
                        #leaving the rook source square empty
                        state_dict['h8'] = 'None'
                        #board.push(move)  # Make the move

                elif board.is_queenside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles queenside')
                        #move the appopriate rook
                        state_dict['d1'] = 'a1'
                        #leaving the rook source square empty
                        state_dict['a1'] = 'None'
                        #board.push(move)  # Make the move

                    elif not board.turn:
                        #print(counter,'... and black castles queenside')
                        #move the appopriate rook
                        state_dict['d8'] = 'a8'
                        #leaving the rook source square empty
                        state_dict['a8'] = 'None'
                        #board.push(move)  # Make the move

                else:
                    #update the piece on the target square
                    state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                    #update the source square to be empty
                    state_dict[chess.square_name(move.from_square)] = None


                    #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

        board.push(move)  # Make the move
        #print(state_dict)
            
    if board.is_checkmate():
        #print("checkmate")
        #do checkmate stuff here

    game_counter += 1
    #print('end of game',game_counter)
    
    total_move_counter += counter
    #print('total moves analyzed',total_move_counter)

    
    #print('board outcome',board.outcome())


    #print('is game over?',board.is_game_over())
    board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

checkmate
end of game 1
total_moves_analyzed 25
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 2
total_moves_analyzed 60
board outcome None
end of game 3
total_moves_analyzed 81
board outcome None
end of game 4
total_moves_analyzed 175
board outcome None
end of game 5
total_moves_analyzed 221
board outcome None
end of game 6
total_moves_analyzed 284
board outcome None
end of game 7
total_moves_analyzed 322
board outcome None
end of game 8
total_moves_analyzed 377
board outcome None
end of game 9
total_moves_analyzed 450
board outcome None
end of game 10
total_moves_analyzed 540
board outcome None
checkmate
end of game 11
total_moves_analyzed 614
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 12
total_moves_analyzed 707
board outcome None
en passant
end of game 13
total_moves_analyzed 756
board outcome None
checkmate
end of game 14
total_moves_analyzed 825
board outcome Outcome(termination=<Termination.CHEC

checkmate
end of game 108
total_moves_analyzed 6780
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 109
total_moves_analyzed 6919
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 110
total_moves_analyzed 7004
board outcome None
end of game 111
total_moves_analyzed 7064
board outcome None
checkmate
end of game 112
total_moves_analyzed 7129
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 113
total_moves_analyzed 7189
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 114
total_moves_analyzed 7306
board outcome None
end of game 115
total_moves_analyzed 7351
board outcome None
checkmate
end of game 116
total_moves_analyzed 7424
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 117
total_moves_analyzed 7481
board outcome None
checkmate
end of game 118
total_moves_analyzed 7537
board 

total_moves_analyzed 13011
board outcome None
end of game 209
total_moves_analyzed 13050
board outcome None
end of game 210
total_moves_analyzed 13122
board outcome None
end of game 211
total_moves_analyzed 13190
board outcome None
end of game 212
total_moves_analyzed 13229
board outcome None
end of game 213
total_moves_analyzed 13262
board outcome None
end of game 214
total_moves_analyzed 13340
board outcome None
end of game 215
total_moves_analyzed 13383
board outcome None
end of game 216
total_moves_analyzed 13420
board outcome None
end of game 217
total_moves_analyzed 13485
board outcome None
checkmate
end of game 218
total_moves_analyzed 13541
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 219
total_moves_analyzed 13582
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 220
total_moves_analyzed 13605
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
e

checkmate
end of game 312
total_moves_analyzed 19627
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 313
total_moves_analyzed 19670
board outcome None
checkmate
end of game 314
total_moves_analyzed 19752
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 315
total_moves_analyzed 19786
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 316
total_moves_analyzed 19813
board outcome None
end of game 317
total_moves_analyzed 19846
board outcome None
end of game 318
total_moves_analyzed 19901
board outcome None
en passant
end of game 319
total_moves_analyzed 19964
board outcome None
end of game 320
total_moves_analyzed 19994
board outcome None
checkmate
end of game 321
total_moves_analyzed 20053
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 322
total_moves_analyzed 20090
board outcome None
checkmate
end of game 323
total_moves_an

end of game 434
total_moves_analyzed 26267
board outcome None
end of game 435
total_moves_analyzed 26279
board outcome None
end of game 436
total_moves_analyzed 26332
board outcome None
end of game 437
total_moves_analyzed 26366
board outcome None
end of game 438
total_moves_analyzed 26428
board outcome None
checkmate
end of game 439
total_moves_analyzed 26488
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 440
total_moves_analyzed 26529
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 441
total_moves_analyzed 26577
board outcome None
end of game 442
total_moves_analyzed 26621
board outcome None
end of game 443
total_moves_analyzed 26664
board outcome None
checkmate
end of game 444
total_moves_analyzed 26766
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
checkmate
end of game 445
total_moves_analyzed 26826
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, win

...

checkmate
end of game 24290
total_moves_analyzed 1610931
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 24291
total_moves_analyzed 1611006
board outcome None
end of game 24292
total_moves_analyzed 1611101
board outcome None
checkmate
end of game 24293
total_moves_analyzed 1611178
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
en passant
end of game 24294
total_moves_analyzed 1611235
board outcome None
end of game 24295
total_moves_analyzed 1611300
board outcome None
checkmate
end of game 24296
total_moves_analyzed 1611387
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 24297
total_moves_analyzed 1611458
board outcome None
end of game 24298
total_moves_analyzed 1611495
board outcome None
end of game 24299
total_moves_analyzed 1611577
board outcome None
end of game 24300
total_moves_analyzed 1611619
board outcome None
en passant
end of game 24301
total_moves_analyzed 1611698
board o

board outcome None
end of game 24400
total_moves_analyzed 1618307
board outcome None
end of game 24401
total_moves_analyzed 1618380
board outcome None
checkmate
end of game 24402
total_moves_analyzed 1618427
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 24403
total_moves_analyzed 1618477
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 24404
total_moves_analyzed 1618652
board outcome Outcome(termination=<Termination.INSUFFICIENT_MATERIAL: 3>, winner=None)
end of game 24405
total_moves_analyzed 1618744
board outcome None
checkmate
end of game 24406
total_moves_analyzed 1618812
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 24407
total_moves_analyzed 1618885
board outcome None
end of game 24408
total_moves_analyzed 1618920
board outcome None
checkmate
end of game 24409
total_moves_analyzed 1619020
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, 

end of game 24504
total_moves_analyzed 1626092
board outcome None
end of game 24505
total_moves_analyzed 1626119
board outcome None
end of game 24506
total_moves_analyzed 1626196
board outcome None
end of game 24507
total_moves_analyzed 1626281
board outcome None
end of game 24508
total_moves_analyzed 1626290
board outcome None
end of game 24509
total_moves_analyzed 1626366
board outcome None
checkmate
end of game 24510
total_moves_analyzed 1626419
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 24511
total_moves_analyzed 1626467
board outcome None
checkmate
end of game 24512
total_moves_analyzed 1626596
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
checkmate
end of game 24513
total_moves_analyzed 1626696
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 24514
total_moves_analyzed 1626723
board outcome None
end of game 24515
total_moves_analyzed 1626749
board outcome None
end of ga

checkmate
end of game 24612
total_moves_analyzed 1633338
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 24613
total_moves_analyzed 1633409
board outcome None
end of game 24614
total_moves_analyzed 1633522
board outcome None
end of game 24615
total_moves_analyzed 1633648
board outcome None
end of game 24616
total_moves_analyzed 1633683
board outcome None
end of game 24617
total_moves_analyzed 1633799
board outcome None
checkmate
end of game 24618
total_moves_analyzed 1633856
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=True)
end of game 24619
total_moves_analyzed 1633966
board outcome None
end of game 24620
total_moves_analyzed 1634005
board outcome None
checkmate
end of game 24621
total_moves_analyzed 1634085
board outcome Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)
end of game 24622
total_moves_analyzed 1634146
board outcome None
end of game 24623
total_moves_analyzed 1634205
board outcome None
end of ga

KeyboardInterrupt: 

In [10]:
print(game_counter,'games analyzed')
print(total_move_counter,'moves analyzed')

24682 games analyzed
1638162 moves analyzed


In [11]:
weights

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,g7,h7,a8,b8,c8,d8,e8,f8,g8,h8
a1,0,12,11,15,0,4,6,22,38,28,...,538,588,2590,795,957,999,0,866,668,1975
b1,9,0,9,2,0,7,8,13,13,12,...,321,257,777,1135,1381,930,0,1895,2661,569
c1,27,19,0,20,0,11,13,33,28,22,...,787,666,576,1828,1,863,0,4305,4049,1008
d1,26,9,15,0,0,9,11,20,24,13,...,1362,1336,1706,2573,3201,5470,0,2269,2259,1700
e1,0,0,1,1,0,0,0,1,0,0,...,104,114,206,195,255,511,0,320,285,257
f1,18,7,8,14,0,0,8,20,14,14,...,371,550,854,3713,4304,681,0,0,2327,605
g1,19,10,14,9,0,5,0,20,12,13,...,678,502,472,3212,1673,862,0,1403,1131,845
h1,23,14,14,12,0,10,7,0,35,22,...,823,880,2005,813,1043,1149,0,1083,969,2393
a2,10,2,3,1,0,3,2,2,0,13,...,27,34,67,340,180,120,0,135,62,43
b2,8,7,5,2,0,8,4,3,31,0,...,17,15,97,299,121,120,0,1442,469,41


In [12]:
a = weights - np.identity(32)
print(a)

        a1      b1      c1      d1   e1      f1      g1      h1      a2  \
a1    -1.0    12.0    11.0    15.0  0.0     4.0     6.0    22.0    38.0   
b1     9.0    -1.0     9.0     2.0  0.0     7.0     8.0    13.0    13.0   
c1    27.0    19.0    -1.0    20.0  0.0    11.0    13.0    33.0    28.0   
d1    26.0     9.0    15.0    -1.0  0.0     9.0    11.0    20.0    24.0   
e1     0.0     0.0     1.0     1.0 -1.0     0.0     0.0     1.0     0.0   
f1    18.0     7.0     8.0    14.0  0.0    -1.0     8.0    20.0    14.0   
g1    19.0    10.0    14.0     9.0  0.0     5.0    -1.0    20.0    12.0   
h1    23.0    14.0    14.0    12.0  0.0    10.0     7.0    -1.0    35.0   
a2    10.0     2.0     3.0     1.0  0.0     3.0     2.0     2.0    -1.0   
b2     8.0     7.0     5.0     2.0  0.0     8.0     4.0     3.0    31.0   
c2     5.0     2.0     2.0     3.0  0.0     7.0     1.0     7.0    14.0   
d2     4.0     3.0     5.0     2.0  0.0     4.0     4.0    11.0    11.0   
e2     6.0     3.0     3.

In [13]:
b = np.zeros(32)
print(b)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:
x = linalg.solve(a, b)
x

array([ 0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
       -0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
       -0., -0., -0.,  0., -0.,  0.])

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [15]:
np.linalg.det(a)

1.9322304165338496e+92

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved with a Matrix Factorization.

In [16]:
eigen_values, eigen_vectors = np.linalg.eig(a)  

In [17]:
values = eigen_vectors[:, np.argmin(eigen_values)] 

In [18]:
## Let's normalize the values

In [19]:
list = []
for i in values:
    list.append(abs(float(i)))

<ipython-input-19-fd238d3eb4fa>:3: ComplexWarning: Casting complex values to real discards the imaginary part
  list.append(abs(float(i)))


In [20]:
#The fractional inverse of the strongest pawn
factor = 1 / np.max(list[8:-8])

In [21]:
normed = [i*factor for i in list]

In [22]:
normalized_list = ["%.2f"%i for i in normed]

In [23]:
print(normalized_list[0:8])
print(normalized_list[8:8+8])
print(normalized_list[8+8:8+8+8])
print(normalized_list[-8:])

['2.10', '2.37', '2.91', '5.59', '0.55', '2.89', '2.50', '2.27']
['0.34', '0.72', '0.72', '0.88', '0.97', '0.71', '0.62', '0.33']
['0.30', '0.78', '0.80', '0.91', '1.00', '0.74', '0.62', '0.36']
['2.17', '2.41', '2.75', '5.48', '0.69', '2.95', '2.40', '2.14']


In [ ]:
board.reset()
game = chess.pgn.read_game(pgn)

for move in game.mainline_moves():
        print(move.uci())
        print('turn?' , board.turn)
        print('is game over?' , board.is_game_over())
        print('from square: ' ,chess.square_name(move.from_square))
        print('to square: ' ,chess.square_name(move.to_square))
        print('drop: ' ,move.drop)
        print('is capture?' , board.is_capture(move))
        print('is en passant capture?' , board.is_en_passant(move))
        print('is kingside castle?' , board.is_kingside_castling(move))
        print('is quenside castle?' , board.is_queenside_castling(move))

        board.push(move)  # Make the move
        print('is_checkmate?' , board.is_checkmate())
        #print('checkers' , board.checkers())
        print('outcome' , board.outcome())

        #print(board)
        #print(board.piece_map())
    #board